title: Ensemble Learning for HR Classification
description: This notebook explores combining multiple ML classifiers for HR applicant selection using ensemble learning techniques to improve predictive performance.

# Development of Ensemble Learning for HR case
**Author:** Juan Guillermo Osío Jaramillo

**Role:** Freelance consultant and developer on Machine Learning and Ensemble Learning

<br>

<a id='introduction'></a>
# Introduction
---
The client has developed a battery of ML classification algorithms to automate the *hiring* process for a Human Resources (HR) department. Each algorithm predicts whether an applicant will be selected at each stage of the selection pipeline from individual-level covariates. The client wonders if one can combine the strengths of the individual classifiers into a *single* system outperforming all of them, something like *the whole being greater than the sum of its parts*. I prompted my client to the topic of ensemble learning, a meta-heuristic comprehending multiple techniques to flexibly combine output from multiple predictors (called base classifiers) into a single one by means of taking *weighted averages* of their predictive signals. I developed a configurable **ensemble algorithm** to combine predictions from multiple base classifiers and discovered a sheer amount of non-singleton ensembles outperforming the singleton ones from an F1 perspective.

<br>

<a id='table-of-contents'></a>
## Table of Contents
---

As this report is served as an online notebook, I will host all the code without giving a full modular structure. Yet the following table of contents glimpses the overall architecture.


0. [Application set up](#set-up)
**PART 1 - Ensemble learning for the stages of HR**
1. [Serving ML samples from HR data](#ml-samples)
2. [Base classifiers](#base-classifiers)
3. [Ensemble learning](#ensemble-learning)
4. [Experimental-grid](#experimental-grid)
5. [Run Experiments](#run-experiments)
**PART 2 - Ensemble learning for the selection pipeline of HR**

<br>

<a id='set-up'></a>
## Aplication set up
---

In [ ]:
#
# 0.0 set-up  libraries
#
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
!pip install psutil
import psutil
import time


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#
# 0.1 set-up a cloud (drive based) filesystem
#

os.chdir(os.path.join('drive','MyDrive', 'professional portfolio','ensemble_learning'))
os.listdir()

['second_data_Set_listo - copia.csv',
 'stage_2.csv',
 'stage_1.csv',
 'filter1_new.csv',
 'Stage_2_results.xlsx',
 'Stage_1_results.xlsx']

# **PART 1 - Ensemble learning for the stages of HR**



<br>

<a id='ml-samples'></a>
## 1. Serving ML samples from HR data
---

The raw data for this exercise is a couple of tabular datasets, each listing individual level covariates for a set of workers (applicants), and a target being 1 if (only if) they susceded in a stage of the selection pipeline.
As always in the Machine Learning development workflow, we need to define the data samples (train and test, X an Y) to enable further model development activities. In this section I implement the pattern of defining a configurable data manager which holds the input dataset, perform minimal validations, and serves featurization and sample splitting in a fell swoop. The manager accepts kwargs-parametrization, so that the caller can adjust some choices at the data processing step. I will instiante the manager for stages (1,2) of the selection process so that we can get ML samples from each.

In [ ]:
#
# (0)
#
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
#
# (1)
#
class HRStage:
    def __init__(self, stage_data, target_name, application_id='enrollee_id'):
        """
        Initialize the HRStage class with stage data, target name, and application ID.

        Parameters:
        stage_data (pd.DataFrame): The data frame containing features and target.
        target_name (str): The name of the target column in stage_data.
        application_id (str): The name of the application ID column in stage_data. Default is 'default_id'.
        """
        stage_data=stage_data.sample(frac=1)
        self._break_if_filtration(stage_data, target_name)
        self.stage_target_name = target_name
        self.stage_target = stage_data[target_name]
        self.stage_data = stage_data.drop(target_name, axis=1).reset_index(drop=True)
        #
        self.application_id = application_id
        self.raw_application_ids = stage_data[application_id].reset_index(drop=True).copy()

    def _break_if_filtration(self, data, target_name):
        target = data[target_name]
        for col in data.columns:
            if col != target_name:
                feature_data = data[col]
                if feature_data.dtype == 'object':
                    feature_data = pd.get_dummies(feature_data, drop_first=True)
                else:
                    feature_data = feature_data.values.reshape(-1, 1)

                model = LinearRegression().fit(feature_data, target)
                predictions = model.predict(feature_data)
                if r2_score(target, predictions) == 1.0:
                    raise ValueError(f"Feature '{col}' perfectly predicts the target. Initialization halted.")

        print("No single feature perfectly predicts the outcome. Proceeding with initialization.")

    def _process_features(self, **processing_parameters):
        """
        Process the features of the dataset with flexible processing parameters.

        Parameters:
        processing_parameters (dict): A dictionary of processing parameters.
                                    Can include 'categorical_method' and 'scale_features'.

        Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: The processed features DataFrame,
                                                        and the separate numerical and categorical DataFrames.
        """
        # Set default values
        categorical_method = processing_parameters.get('categorical_method', 'label_encoding')
        scale_features = processing_parameters.get('scale_features', True)

        # Separate numerical and categorical features
        numerical_df = self.stage_data.select_dtypes(include=['int64', 'float64'])
        categorical_df = self.stage_data.select_dtypes(include=[object])

        # Process categorical features
        if categorical_method == "label_encoding":
            for col in categorical_df.columns:
                le = LabelEncoder()
                categorical_df[col] = le.fit_transform(categorical_df[col])
        elif categorical_method == "one-hot-encoding":
            categorical_df = pd.get_dummies(categorical_df)
        else:
            raise ValueError("Invalid categorical_method. Choose 'label_encoding' or 'one-hot-encoding'.")

        if scale_features:
            # Scaling numerical features
            scaler = StandardScaler()
            numerical_df = pd.DataFrame(scaler.fit_transform(numerical_df), columns=numerical_df.columns)

        # Merge numerical and categorical features
        processed_df = pd.concat([numerical_df, categorical_df], axis=1)

        return processed_df

    def serve_stage_samples(self,
                            test_ratio=0.5,
                            negative_to_positive_ratio=None,
                            test_id_applications=None,
                            **processing_parameters):

        # Process the features using processing parameters
        processed_features = self._process_features(**processing_parameters)

        # Add target column back to the dataset for splitting
        processed_data = processed_features.copy()
        processed_data.reset_index(drop=True, inplace=True)
        self.stage_target.reset_index(drop=True, inplace=True)
        processed_data[self.stage_target_name] = self.stage_target

        if test_id_applications is not None:
            # Use raw_application_ids to split data
            test_mask = self.raw_application_ids.isin(test_id_applications)
            test_data = processed_data[test_mask]
            train_data = processed_data[~test_mask]

            # Extract the application IDs actually found in the test data
            test_ids_found = self.raw_application_ids[test_mask]
        else:
            # Random splitting
            train_data, test_data = train_test_split(processed_data, test_size=test_ratio, random_state=42)

            # Extract the application IDs from the randomly created test set
            test_ids_found = self.raw_application_ids[test_data.index]

        # Adjust training data based on negative_to_positive_ratio
        if negative_to_positive_ratio is not None:
            pos_subset = train_data[train_data[self.stage_target_name] == 1]
            neg_subset = train_data[train_data[self.stage_target_name] == 0]
            num_pos_cases = pos_subset.shape[0]
            num_neg_cases = int(negative_to_positive_ratio * num_pos_cases)
            neg_subset = neg_subset.sample(n=min(num_neg_cases, neg_subset.shape[0]), random_state=42)
            train_data = pd.concat([pos_subset, neg_subset]).sample(frac=1, random_state=42)

        # Separate features and target for training and testing sets
        X_train = train_data.drop(self.stage_target_name, axis=1)
        y_train = train_data[self.stage_target_name]
        X_test = test_data.drop(self.stage_target_name, axis=1)
        y_test = test_data[self.stage_target_name]

        return X_train, X_test, y_train, y_test, [_ for _ in test_ids_found]

In [ ]:
#
# (2.1)
#
stage_1_df= pd.read_csv('stage_1.csv', sep=",")
stage_1_df=stage_1_df.sample(frac=.2)
stage_1_df["Target"] = stage_1_df["Target"].astype('int64')
stage_1_df.head()
#
hr_st1=HRStage(stage_1_df, target_name="Target")
hr_st1.stage_data.info()

No single feature perfectly predicts the outcome. Proceeding with initialization.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4882 entries, 0 to 4881
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           4882 non-null   int64  
 1   enrollee_id          4882 non-null   int64  
 2   gender               4882 non-null   int64  
 3   education_level      4882 non-null   object 
 4   major_discipline     4882 non-null   object 
 5   recruitment_channel  4882 non-null   float64
 6   applied_before       4882 non-null   float64
 7   Vacant               4882 non-null   object 
 8   disability           4882 non-null   float64
 9   offered              4882 non-null   float64
 10  experience           4882 non-null   float64
dtypes: float64(5), int64(3), object(3)
memory usage: 419.7+ KB


In [ ]:
round(hr_st1.stage_target.value_counts(normalize=True)*100, 3)

,proportion
Target,
0,72.962
1,27.038


In [ ]:
#
# (2.2)
#
stage_2_df= pd.read_csv('stage_2.csv', sep=";")
hr_st2=HRStage(stage_2_df, target_name="tarjet")
hr_st2.stage_data.info()

No single feature perfectly predicts the outcome. Proceeding with initialization.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17725 entries, 0 to 17724
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   enrollee_id     17725 non-null  int64 
 1   Vacant          17725 non-null  object
 2   extra_lidr      17725 non-null  int64 
 3   big_5_1         17725 non-null  int64 
 4   big_5_2         17725 non-null  int64 
 5   big_5_3         17725 non-null  int64 
 6   big_5_4         17725 non-null  int64 
 7   big_5_5         17725 non-null  int64 
 8   drugs_C         17725 non-null  int64 
 9   hard_skill_1    17725 non-null  int64 
 10  hard skill_2    17725 non-null  int64 
 11  hard skill_3    17725 non-null  int64 
 12  dt_m            17725 non-null  int64 
 13  dt_n            17725 non-null  int64 
 14  dt_p            17725 non-null  int64 
 15  interview       17725 non-null  int64 
 16  ref_ck      

In [ ]:
round(hr_st2.stage_target.value_counts(normalize=True)*100, 3)

,proportion
tarjet,
0,99.278
1,0.722


<br>

<a id='base-classifiers'></a>
## 2. Base classifiers
---

In the simplest approach to development, each sample defines an *independent* classification problem. We will sample from the plethora of ML predictive algorithm for the detection of predictive patterns. My client suggested and implemented the following selection of ML algorithms. I will *abstract away* implementation details and standardize behind a common interface, thus facilitating further model assestment activities. As oftentimes EL combine from the underlyng probabilities, I will standardize this model to yield a probability model, instead of a binary vector. Each function host a distintive ML algorithm and will perform a full fit-predict sample from train and test samples, then returning a vector of probabilties. The algorithms will become the basis of the overarching solutions, thus being named *base classifier* in the ensemble learning literature.



In [ ]:
#
# 0.1
#
# Import classifiers from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder

# Import deep learning models from TensorFlow/Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Additional utilities
import warnings
from sklearn.exceptions import ConvergenceWarning
import numpy as np


In [ ]:
#
# 0.2
#
warnings.filterwarnings("ignore", category=ConvergenceWarning)
#
def probs_with_random_forest_classifier(X_train, y_train, X_test, **rf_params):
    # Create and train the RandomForest model
    rf = RandomForestClassifier(**rf_params)
    rf.fit(X_train, y_train)

    # Return probability estimates for the test set
    return rf.predict_proba(X_test)[:, 1]  # Assuming binary classification
#
def probs_with_svc(X_train, y_train, X_test, **svc_params):
    # Create and train the SVC model
    # Ensure to set probability=True for probability estimates
    svc = SVC(probability=True, **svc_params)
    svc.fit(X_train, y_train)

    # Return probability estimates for the positive class in the test set
    return svc.predict_proba(X_test)[:, 1]  # Assuming binary classification
#
def probs_with_logistic_regression(X_train, y_train, X_test, **lr_params):
    # Create and train the Logistic Regression model
    lr = LogisticRegression(**lr_params)
    lr.fit(X_train, y_train)

    # Return probability estimates for the positive class in the test set
    return lr.predict_proba(X_test)[:, 1]
#
def probs_with_knn(X_train, y_train, X_test, **knn_params):
    # Create and train the KNN model
    knn = KNeighborsClassifier(**knn_params)
    knn.fit(X_train, y_train)

    # Return probability estimates for the positive class in the test set
    return knn.predict_proba(X_test)[:, 1]  # Assuming binary classification
#
def probs_with_DNN(X_train, y_train, X_test, **dnn_params):
    # Ensure data is in correct type
    X_train = np.asarray(X_train).astype('float32')
    y_train = np.asarray(y_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')

    # If y_train is not numeric, convert it
    if y_train.dtype == 'object':
        label_encoder = LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)
    y_train = y_train.astype('float32')

    # Default hyperparameters
    default_params = {
        'epochs': 5,
        'batch_size': 25,
        'input_dim': X_train.shape[1],
        'first_layer_units': 64,
        'second_layer_units': 32,
        'dropout_rate': 0.2,
        'activation': 'relu',
        'output_activation': 'sigmoid',
        'loss': 'binary_crossentropy',
        'optimizer': 'adam'
    }

    # Update defaults with any provided parameters
    default_params.update(dnn_params)

    # Model definition
    model = Sequential()
    model.add(Dense(default_params['first_layer_units'], input_dim=default_params['input_dim'], activation=default_params['activation']))
    model.add(Dropout(default_params['dropout_rate']))
    model.add(Dense(default_params['second_layer_units'], activation=default_params['activation']))
    model.add(Dropout(default_params['dropout_rate']))
    model.add(Dense(1, activation=default_params['output_activation']))  # Binary classification

    # Compile model
    model.compile(loss=default_params['loss'], optimizer=default_params['optimizer'], metrics=['accuracy'])

    # Model training
    model.fit(X_train, y_train, epochs=default_params['epochs'], batch_size=default_params['batch_size'], verbose=0)

    # Return probability estimates for the positive class in the test set
    return model.predict(X_test, verbose=0).flatten()

#
def probs_with_MLPClassifier(X_train, y_train, X_test, **mlp_params):
    # Create and train the MLPClassifier model with additional parameters
    ann2 = MLPClassifier(**mlp_params, max_iter=300, random_state=42)
    ann2.fit(X_train, y_train)

    # Return probability estimates for the positive class in the test set
    return ann2.predict_proba(X_test)[:, 1]  # Assuming binary classification
#
BASE_PROB_MODELS = {
    "RF":probs_with_random_forest_classifier,
    "SVC":probs_with_svc,
    "LR":probs_with_logistic_regression,
    "KNN":probs_with_knn,
    "DNN":probs_with_DNN,
    "MLP":probs_with_MLPClassifier
}


<br>

<a id='ensemble-learning'></a>
## 3. Ensemble learning
---

Ensemble Learning comprehends a wide array of techniques to combine inferences (predictions) from the base classifiers into a single predictive system. There are a few hyperparameters involved:

- **Weight vector** to weight each predictive signal.
- **Voting strategy**:
  - *Soft voting*: Averaging probabilities.
  - *Hard voting*: Taking majority votes on the predictions.

The following programs implement the same fit-predict cycle for a particular model ensemble (set of weights). That is, we predict from the base classifiers and get the predictions back to the caller. Then again, configuration for the ensembled prediction is received at runtime, thus inviting further calibration.




In [ ]:
#
# decorator to track consumption of computational resources
#
import time
import psutil
import functools

def check_resources(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        process = psutil.Process()
        time_start = time.time()
        cpu_times_start = process.cpu_times()
        ram_start = process.memory_info().rss

        # Execute the wrapped function
        result = func(*args, **kwargs)

        time_end = time.time()
        cpu_times_end = process.cpu_times()
        ram_end = process.memory_info().rss

        # Construct the resource usage report
        resource_report = {
            'time_taken': time_end - time_start,
            'cpu_user_time': cpu_times_end.user - cpu_times_start.user,
            'cpu_system_time': cpu_times_end.system - cpu_times_start.system,
            'ram_used': ram_end - ram_start
        }

        return result, resource_report

    return wrapper

In [ ]:
import logging
from sklearn.metrics import roc_auc_score

# Set up the logging configuration
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@check_resources
def voting_ensemble(base_learners, X_train, y_train, X_test, weights="all_identical", threshold=0.5, voting_type='soft'):
    try:
        # Determine effective weights based on the specified method
        if weights == "all_identical":
            effective_weights = [1 / len(base_learners)] * len(base_learners)

        elif weights == "self_moderated":
            auc_scores = []
            for learner in base_learners:
                preds = learner(X_train, y_train, X_train)
                auc = roc_auc_score(y_train, preds)
                auc_scores.append(auc)
            total_auc = sum(auc_scores)
            effective_weights = [auc / total_auc for auc in auc_scores]

        elif isinstance(weights, list):
            if len(weights) != len(base_learners):
                raise ValueError("Length of weights must match number of base learners.")
            if sum(weights) != 1:
                raise ValueError("Weights must sum to 1.")
            effective_weights = weights

        # Filtering base learners and weights with non-zero weights
        effective_base_learners = [learner for learner, weight in zip(base_learners, effective_weights) if weight != 0]
        all_effective_weights = effective_weights
        effective_weights = [weight for weight in effective_weights if weight != 0]

        # Ensuring there are base learners after filtering
        if not effective_base_learners:
            raise ValueError("No base learners are available after filtering by weights.")

        # Count of effective learners
        effective_learner_count = len(effective_base_learners)

        # Generating predictions from effective base learners
        predictions = []
        for learner, weight in zip(effective_base_learners, effective_weights):
            preds = learner(X_train, y_train, X_test)
            if voting_type == 'soft':
                predictions.append(preds * weight)
            else:
                predictions.append(preds > threshold)

        # Aggregating predictions based on voting type
        if voting_type == 'soft':
            avg_prediction = sum(predictions)
            final_prediction = (avg_prediction > threshold).astype(int)
        else:
            final_prediction = sum(predictions)
            # Adjust majority voting threshold based on effective learner count
            majority_threshold = effective_learner_count // 2
            final_prediction = (final_prediction > majority_threshold).astype(int)

        # Return the final prediction along with ensemble parameters
        return {
            'final_prediction': final_prediction,
            'ensemble_parameters': {
                'voting_type': voting_type,
                'threshold': threshold,
                'weights': all_effective_weights
            }
        }

    except Exception as e:
        # Log the error with parameters
        logger.error("An error occurred during voting ensemble computation.", exc_info=True)
        logger.error("Parameters: base_learners=%s, weights=%s, threshold=%s, voting_type=%s",
                     base_learners, weights, threshold, voting_type)
        raise

# Example usage:
# voting_ensemble([...], X_train, y_train, X_test)


<br>

<a id='experimental-grid'></a>
## 4. Experimental-grid
---

From an architectural POV, we are close to running predictions on the test data. A concrete run of the predictive system requires configuration for the featurization (performed by the data manager) and for the ensemble itself (performed by the ensembler). In order to support wide experimtentation and system calibration by means of grid-search, the following piece of code defines a grid of alternative configurations to be tested upon.



In [ ]:
#
# O. set-up of a parameter grid for ensemble lerning
#
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, f1_score
from itertools import combinations
#
# (1) Base learners
#
BASE_PROB_MODELS = {
    "RF":probs_with_random_forest_classifier,
    "SVC":probs_with_svc,
    "LR":probs_with_logistic_regression,
    "KNN":probs_with_knn,
    "DNN":probs_with_DNN,
    "MLP":probs_with_MLPClassifier
}

#
# (2) evaluation data
#

categorical_methods = ['label_encoding', 'one-hot-encoding']
scale_options = [True, False]

#
# (3.1) weight scheme
#

num_base_models = len(BASE_PROB_MODELS)
weight_schemes = ['all_identical', 'self_moderated']

# Add weight schemes for individual models (singletons)
# Here, each model gets the entire weight of 1.
weight_schemes += [[1 if i == j else 0 for i in range(num_base_models)] for j in range(num_base_models)]

#weight_schemes =[]

# Add weight schemes for pairs of models (couples)
# Each model in the pair gets a weight of 0.5.
for i, j in combinations(range(num_base_models), 2):
    weights = [0] * num_base_models
    weights[i] = 0.5
    weights[j] = 0.5
    weight_schemes.append(weights)

#
# (3.2) other ensemble parameters
#

thresholds = [0.25, 0.5, 0.75]
voting_types = ['soft', 'hard']
negative_to_positive_ratios=[None, 1, 3]

#
# (5) Define the ensemble parameter grid
#

parameter_grid = []
for weights in weight_schemes:
    for threshold in thresholds:
        for voting_type in voting_types:
            # For each combination of ensemble parameters, add all combinations of preprocessing parameters
            for cat_method in categorical_methods:
                for scale in scale_options:
                    for negative_ratio in negative_to_positive_ratios:
                        parameter_grid.append({
                            'weights': weights,
                            'threshold': threshold,
                            'voting_type': voting_type,
                            'categorical_method': cat_method,
                            'scale_features': scale,
                            "negative_to_positive_ratio":negative_ratio
                        })

# Print the total number of combinations in the full grid
len(parameter_grid)

1656


<br>

<a id='run-experiments'></a>
## 5. Run Experiments
---

We are in position now to run experimentation over the hyperparameter grid as previosly defined. The program *perform_custom_grid_search(...)* will accumulate performance metrics for all the configurations in the defined grid. The prediction returned for each configuration will be compared with the golden truth, given by the test vector of the stage specific data. The performance metric per configuration will be exported to a disk for futher use.

In [ ]:
#
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import precision_score, f1_score, recall_score, classification_report
#
import pandas as pd
from sklearn.metrics import precision_score, f1_score
from random import shuffle
# Ignore UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [ ]:
#
# (7) running experiments
#
def perform_custom_grid_search( hr_stage, parameter_grid, label, export_frequency=50):
    # Define the parse_weights_to_string function inside
    def parse_weights_to_string(weights):
        if isinstance(weights, str):
            return weights
        weight_strs = []
        for model_key, weight in zip(BASE_PROB_MODELS.keys(), weights):
            if weight > 0:
                weight_percent = round(weight * 100, 4)
                weight_strs.append(f"{weight_percent}% {model_key}")
        return " + ".join(weight_strs)

    # Generate a file name using the label
    file_name_template = f"{label}_results.xlsx"

    # Shuffle the parameter grid
    #parameter_grid=random.choi
    results = []

    shuffle(parameter_grid)

    # Iterate over the parameter grid
    for idx, params in enumerate(parameter_grid):

        # Sample preparation and model training
        X_train, X_test, y_train, y_test, _ = hr_stage.serve_stage_samples(
            test_ratio=0.5,
            negative_to_positive_ratio=params['negative_to_positive_ratio'],
            categorical_method=params['categorical_method'],
            scale_features=params['scale_features']
        )

        ensemble_result, resources = voting_ensemble(
            base_learners=list(BASE_PROB_MODELS.values()),
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            weights=params['weights'],
            threshold=params['threshold'],
            voting_type=params['voting_type']
        )

        # Model evaluation
        predictions = ensemble_result['final_prediction']
        precision = precision_score(y_test, predictions, average='binary', zero_division=0)
        f1 = f1_score(y_test, predictions, average='binary', zero_division=0)
        recall = recall_score(y_test, predictions, average='binary', zero_division=0)
        report = classification_report(y_test, predictions, output_dict=True)
        support = report['1']['support']  # Assuming '1' is the positive class

        # Parsing ensemble weights for readability
        weights_description = parse_weights_to_string(ensemble_result['ensemble_parameters']['weights'])
        # Constructing the result dictionary
        result = {
            'weights': weights_description,
            'categorical_method': params['categorical_method'],
            'scale_features': params['scale_features'],
            'negative_to_positive_ratio': params['negative_to_positive_ratio'],
            'threshold': round(params['threshold'], 4),
            'voting_type': params['voting_type'],
            'precision': round(precision, 4),
            'f1_score': round(f1, 4),
            'recall': round(recall, 4),
            'support': support
        }

        result.update(resources)
        print(result)
        results.append(result)

        # Periodic export of results
        if (idx + 1) % export_frequency == 0 or (idx + 1) == len(parameter_grid):
            pd.DataFrame(results).to_excel(file_name_template)
            print(f"Results exported to {file_name_template} after processing {idx + 1} models.")

    # Final export outside the loop
    pd.DataFrame(results).to_excel(file_name_template)
    print(f"Final results exported to {file_name_template}.")

In [ ]:
#
perform_custom_grid_search(hr_st2, parameter_grid,  "Stage_2")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'weights': '50.0% RF + 50.0% DNN', 'categorical_method': 'one-hot-encoding', 'scale_features': True, 'negative_to_positive_ratio': None, 'threshold': 0.75, 'voting_type': 'soft', 'precision': 1.0, 'f1_score': 0.7255, 'recall': 0.5692, 'support': 65.0, 'time_taken': 12.853696584701538, 'cpu_user_time': 7.83, 'cpu_system_time': 0.28, 'ram_used': 45842432}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'weights': '50.0% KNN + 50.0% DNN', 'categorical_method': 'one-hot-encoding', 'scale_features': False, 'negative_to_positive_ratio': 3, 'threshold': 0.75, 'voting_type': 'hard', 'precision': 0.0, 'f1_score': 0.0, 'recall': 0.0, 'support': 65.0, 'time_taken': 2.480496883392334, 'cpu_user_time': 2.3000000000000007, 'cpu_system_time': 0.18999999999999995, 'ram_used': 15687680}
{'weights': '50.0% SVC + 50.0% MLP', 'categorical_method': 'label_encoding', 'scale_features': True, 'negative_to_positive_ratio': None, 'threshold': 0.75, 'voting_type': 'hard', 'precision': 1.0, 'f1_score': 0.6869, 'recall': 0.5231, 'support': 65.0, 'time_taken': 6.497917652130127, 'cpu_user_time': 4.100000000000001, 'cpu_system_time': 3.46, 'ram_used': 7499776}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

{'weights': '50.0% KNN + 50.0% DNN', 'categorical_method': 'label_encoding', 'scale_features': True, 'negative_to_positive_ratio': 1, 'threshold': 0.75, 'voting_type': 'hard', 'precision': 0.1977, 'f1_score': 0.2252, 'recall': 0.2615, 'support': 65.0, 'time_taken': 3.717061996459961, 'cpu_user_time': 2.949999999999818, 'cpu_system_time': 0.020000000000095497, 'ram_used': 1245184}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'weights': '50.0% KNN + 50.0% DNN', 'categorical_method': 'label_encoding', 'scale_features': True, 'negative_to_positive_ratio': 1, 'threshold': 0.25, 'voting_type': 'soft', 'precision': 0.0095, 'f1_score': 0.0188, 'recall': 1.0, 'support': 65.0, 'time_taken': 2.2723026275634766, 'cpu_user_time': 2.1700000000000728, 'cpu_system_time': 0.029999999999972715, 'ram_used': 2707456}
{'weights': '50.0% KNN + 50.0% MLP', 'categorical_method': 'one-hot-encoding', 'scale_features': True, 'negative_to_positive_ratio': 1, 'threshold': 0.75, 'voting_type': 'hard', 'precision': 0.1715, 'f1_score': 0.2928, 'recall': 1.0, 'support': 65.0, 'time_taken': 0.33002400398254395, 'cpu_user_time': 0.34999999999990905, 'cpu_system_time': 0.2300000000000182, 'ram_used': 0}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'weights': '50.0% KNN + 50.0% DNN', 'categorical_method': 'one-hot-encoding', 'scale_features': True, 'negative_to_positive_ratio': None, 'threshold': 0.75, 'voting_type': 'hard', 'precision': 1.0, 'f1_score': 0.0303, 'recall': 0.0154, 'support': 65.0, 'time_taken': 5.238853454589844, 'cpu_user_time': 6.089999999999691, 'cpu_system_time': 0.20000000000004547, 'ram_used': 3055616}
{'weights': '50.0% RF + 50.0% LR', 'categorical_method': 'label_encoding', 'scale_features': False, 'negative_to_positive_ratio': 3, 'threshold': 0.25, 'voting_type': 'soft', 'precision': 0.1416, 'f1_score': 0.2481, 'recall': 1.0, 'support': 65.0, 'time_taken': 0.20963263511657715, 'cpu_user_time': 0.21000000000003638, 'cpu_system_time': 0.0, 'ram_used': 0}
{'weights': '50.0% RF + 50.0% MLP', 'categorical_method': 'label_encoding', 'scale_features': False, 'negative_to_positive_ratio': None, 'threshold': 0.75, 'voting_type': 'hard', 'precision': 0.0, 'f1_score': 0.0, 'recall': 0.0, 'support': 65.0, 'time_take

In [ ]:
#
perform_custom_grid_search(hr_st1, parameter_grid,  "Stage_1")

<br>

<a id='part-2'></a>
# **PART 2  - Ensemble learning for the overall selection pipeline**
---
Create a combination of ensembles to predict results on the total selection process


<br>

<a id='my revision goes here. hide this cells'></a>
## My revision goes here. please ommit so far.
---

Now we will need an abstraction over a selection pipeline with multiple stages.

In [ ]:
#
# 2. class to evaluate predictions for a complete HR process
#

In [ ]:
# Import core libraries
import numpy as np
import pandas as pd
import time
import warnings

# Import classifiers from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Import metrics and model selection tools
from sklearn.metrics import precision_score, f1_score
from sklearn.model_selection import train_test_split

# Import utility functions
from functools import lru_cache

In [ ]:
class HRProcess:
    def __init__(
            self,
            stages,
            application_id,
            test_ratio=0.5
            ):
        """
        Initialize the HRProcess class with a list of stage instances and an application ID.

        Parameters:
        stages (list): A list of HRStage instances.
        application_id (str): The ID used to link records across stages.
        test_ratio (float): The proportion of the dataset to include in the test split.
        """
        self._validate_stages(stages)
        self.stages = stages
        self.application_id = application_id

        # Split unique application IDs into training and test sets
        self.train_ids, self.test_ids = self._split_application_ids(test_ratio)

    def _validate_stages(self, stages):
        """
        Validate that all items in stages are instances of HRStage.

        Parameters:
        stages (list): A list of HRStage instances.

        Raises:
        ValueError: If any item in stages is not an instance of HRStage.
        """
        if not all(isinstance(stage, HRStage) for stage in stages):
            raise ValueError("All stages must be instances of HRStage.")

    def _split_application_ids(self, test_ratio):
        """
        Split the set of unique application IDs into training and validation sets.

        Parameters:
        test_ratio (float): The proportion of the dataset to include in the test split.

        Returns:
        Tuple[set, set]: Sets of training and test application IDs.
        """
        unique_ids = self.stages[0].stage_data[self.application_id].unique()
        train_ids, test_ids = train_test_split(unique_ids, test_size=test_ratio, random_state=42)
        return set(train_ids), set(test_ids)

    def _fit_predict(self, stage, **stage_pred_params):
        """
        Fit a predictive model to a stage and make predictions on the test data.

        Parameters:
        stage (HRStage): The stage to fit the model to.
        stage_pred_params (dict): Additional parameters for prediction.

        Returns:
        dict: A dictionary containing prediction parameters, final predictions, and the corresponding test sample IDs.
        """
        # Warning about the illustrative purpose of this method
        warnings.warn(
            "The _fit_predict method currently uses Logistic Regression for illustrative purposes "
            "and is intended to be replaced with a more robust model or tailored logic.",
            UserWarning
        )

        # Get training and testing data along with the respective IDs
        X_train, X_test, y_train, _ , test_ids = stage.serve_stage_samples(test_id_applications=self.test_ids)

        # Initialize and fit the logistic regression model
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)

        # Make predictions on the test data
        predictions = model.predict(X_test)

        return {
            'parameters': stage_pred_params,
            'final_prediction': predictions,
            'test_sample_ids': test_ids
        }

    #
    #
    #
    def total_evaluation(self, stage_pred_params):
        """
        Evaluate the performance for a specific configuration on the whole selection process.

        Parameters:
        stage_pred_params (dict or list): A dictionary or list containing stage-specific prediction parameters.

        Returns:
        dict: A dictionary containing the evaluated parameters and the scores of the model.
        """
        # Check if stage_pred_params is a list and has the correct length
        if isinstance(stage_pred_params, list) and len(stage_pred_params) != len(self.stages):
            raise ValueError(f"The length of stage_pred_params must match the number of stages. Expected {len(self.stages)}, got {len(stage_pred_params)}.")

        evaluations = {}
        final_predictions_df = pd.DataFrame(index=[_ for _ in self.test_ids] )
        y_test_df = pd.DataFrame(index=[_ for _ in self.test_ids] )

        print( y_test_df.shape)
        print( final_predictions_df.shape)

        for i, stage in enumerate(self.stages):
            # Determine the parameters for the current stage
            current_stage_params = stage_pred_params[i] if isinstance(stage_pred_params, list) else stage_pred_params
            model_result = self._fit_predict(stage, **current_stage_params)

            # Create a DataFrame for current stage predictions
            predictions_df = pd.DataFrame(model_result['final_prediction'], index=model_result['test_sample_ids'], columns=[f'stage_{i}'])
            final_predictions_df = final_predictions_df.join(predictions_df, how='left').fillna(0)

            # Fetch the target variable and application ID from the stage_data attribute
            stage_target_df = pd.DataFrame({
                'application_id': stage.stage_data[ stage.application_id ],
                'target': stage.stage_target
                })
            stage_target_df.set_index( 'application_id', inplace=True)
            stage_target_df=stage_target_df.rename(columns={'target': f'stage_{i}'})

            # Merge the target data for the current stage
            y_test_df = y_test_df.join(stage_target_df, how='left')


        print( y_test_df.shape)
        print( final_predictions_df.shape)


        # Impute missing values in y_test with zeros and take the min
        y_test_final = y_test_df.fillna(0).min(axis=1)

        # Min-summarize predictions on a row basis
        final_predictions = final_predictions_df.min(axis=1)

        # Calculate total precision and F1 score
        total_precision = precision_score(y_test_final, final_predictions, average='binary', zero_division=0)
        total_f1_score = f1_score(y_test_final, final_predictions, average='binary')

        print(pd.Series(y_test_final).value_counts())

        # Add total scores to the evaluations dictionary
        evaluations['total_precision'] = total_precision
        evaluations['total_f1_score'] = total_f1_score
        return evaluations

class HRProcessWEnsembles(HRProcess):
    def __init__(self, stages, application_id, base_prob_models, test_ratio=0.5):
        super().__init__(stages, application_id, test_ratio)
        self.base_prob_models = base_prob_models
        self.final_predictions_df = pd.DataFrame(index=[_ for _ in self.test_ids] )  # DataFrame indexed with test IDs

    def _validate_params(self, params, stage_index):
        """
        Validate the parameters for a given stage.

        Parameters:
        params (dict): Parameters for the stage.
        stage_index (int): Index of the stage.

        Raises:
        ValueError: If any required key is missing in the params.
        """
        required_keys = ['negative_to_positive_ratio', 'categorical_method',
                         'scale_features', 'weights', 'threshold', 'voting_type']

        missing_keys = [key for key in required_keys if key not in params]
        if missing_keys:
            raise ValueError(f"Missing keys {missing_keys} in parameters for stage {stage_index}")

    @lru_cache(maxsize=None)
    def _total_process_outcome(self):
        """
        Compute the final outcome (y_test_final) based on the data of all stages.
        """
        y_test_df = pd.DataFrame(index=[_ for _ in self.test_ids] )
        for i, stage in enumerate(self.stages):
            stage_target_df = pd.DataFrame({
                'application_id': stage.stage_data[stage.application_id],
                'target': stage.stage_target
            })
            stage_target_df.set_index('application_id', inplace=True)
            stage_target_df = stage_target_df.rename(columns={'target': f'stage_{i}'})
            y_test_df = y_test_df.join(stage_target_df, how='left')

        y_test_final = y_test_df.fillna(0).min(axis=1)
        return y_test_final

    def _fit_predict(self, hr_stage, params):
        # Start timing for model training and prediction
        model_start_time = time.time()

        # Sample preparation
        X_train, X_test, y_train, y_test, test_ids = hr_stage.serve_stage_samples(
            test_id_applications=self.test_ids,
            test_ratio=0.5,
            negative_to_positive_ratio=params['negative_to_positive_ratio'],
            categorical_method=params['categorical_method'],
            scale_features=params['scale_features']
        )

        # Assuming voting_ensemble is a predefined function
        ensemble_result, resources = voting_ensemble(
            base_learners=list(self.base_prob_models.values()),
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            weights=params['weights'],
            threshold=params['threshold'],
            voting_type=params['voting_type']
        )

        # Extracting predictions
        predictions = ensemble_result['final_prediction']

        # Calculate precision and F1 score
        precision = precision_score(y_test, predictions, average='binary', zero_division=0)
        f1 = f1_score(y_test, predictions, average='binary')

        # End timing for model training and prediction
        model_end_time = time.time()
        training_prediction_time = model_end_time - model_start_time

        return {
            'final_prediction': predictions,
            'test_ids_used': test_ids,
            'precision': round(precision, 4),
            'f1_score': round(f1, 4),
            'training_prediction_time': round(training_prediction_time, 4)
        }

    def _parse_weights_to_string(self, weights):
        """
        Convert weight vector to a descriptive string.

        Parameters:
        weights (list or str): Weight vector or string representation of weights.

        Returns:
        str: A string representation of the weights.
        """
        if isinstance(weights, str):
            return weights
        weight_strs = []
        for model_key, weight in zip(BASE_PROB_MODELS.keys(), weights):
            if weight > 0:
                weight_percent = round(weight * 100, 4)
                weight_strs.append(f"{weight_percent}% {model_key}")
        return " + ".join(weight_strs)

    def total_evaluation(self, stage_params_list):
        if len(stage_params_list) != len(self.stages):
            raise ValueError(f"Number of parameter sets must match the number of stages. Expected {len(self.stages)}, got {len(stage_params_list)}.")

        # Reset the final predictions DataFrame at the beginning of each evaluation
        self.final_predictions_df = pd.DataFrame(index=[_ for _ in self.test_ids] )  # DataFrame indexed with test IDs

        all_results = {}  # Dictionary to hold all results

        for i, stage_params in enumerate(stage_params_list):
            self._validate_params(stage_params, i)
            stage_evaluations = self._fit_predict(self.stages[i], stage_params)

            # Summarize weights in stage parameters
            stage_params['weights_description'] = self._parse_weights_to_string(stage_params['weights'])

            predictions_df = pd.DataFrame(stage_evaluations['final_prediction'], index=stage_evaluations['test_ids_used'], columns=[f'stage_{i}'])
            self.final_predictions_df = self.final_predictions_df.join(predictions_df, how='left').fillna(0)

            # Append stage-specific parameters and results to all_results
            all_results[f'stage_{i}_params'] = stage_params
            all_results[f'stage_{i}_results'] = stage_evaluations

        # Compute the total process outcome and evaluation
        y_test_final = self._total_process_outcome()
        final_predictions = self.final_predictions_df.min(axis=1)

        # Model Evaluation
        total_precision = precision_score(y_test_final, final_predictions, average='binary', zero_division=0)
        total_f1_score = f1_score(y_test_final, final_predictions, average='binary')
        total_recall = recall_score(y_test_final, final_predictions, average='binary')
        #
        report = classification_report(y_test_final, final_predictions, output_dict=True)
        print(report.keys())
        total_support = report['1.0']['support']  # Assuming binary classification and interested in class '1'

        # Add total process results to all_results
        all_results['total_process'] = {
            'total_precision': round(total_precision, 4),
            'total_f1_score': round(total_f1_score, 4),
            'total_recall': round(total_recall, 4),
            'total_support': total_support
        }

        return all_results



In [ ]:

hr_process = HRProcess([hr_st1, hr_st2], 'enrollee_id')
#
hr_process.total_evaluation(
    stage_pred_params={
        "param1":"val1",
        "param2":"val2",
        "paramN":"valN"
        })

In [ ]:
#
# 3. base learners to learn a probability model
#

In [ ]:
# - BASE_PROB_MODELS was defined before

In [ ]:
#
# 4. ensemble learning (base clasisifier were preloaded in environment)
#
BASE_PROB_MODELS = {
    "RF":probs_with_random_forest_classifier,
    "SVC":probs_with_svc,
    "LR":probs_with_logistic_regression,
    "KNN":probs_with_knn,
    "DNN":probs_with_DNN,
    "MLP":probs_with_MLPClassifier
}

In [ ]:
#
# 5. class to predict the whole model W ensemble learning
#

In [ ]:

#
# (0)
#
stages=[hr_st1, hr_st2]
hr_process_w_ensembles =  HRProcessWEnsembles([hr_st1, hr_st2], 'enrollee_id',base_prob_models=BASE_PROB_MODELS)
#
# (1)
#
test_case_1_params = {
    'weights': "all_identical",  # all_identical weights
    'threshold': 0.5,
    'voting_type': 'soft',
    'categorical_method': 'label_encoding',
    'scale_features': False,
    'negative_to_positive_ratio': 1
}
#
# (2)
#
test_case_1_param_list = [test_case_1_params] * len(stages)  # Assuming 'stages' is defined in your HRProcessWEnsembles instance
#
# (3)
#
results_case_1 = hr_process_w_ensembles.total_evaluation(test_case_1_param_list)
results_case_1

In [ ]:
#
# 6. running the final experiment
#

In [ ]:
#
# (-1) Base learners
#
BASE_PROB_MODELS = {
    "RF":probs_with_random_forest_classifier,
    "SVC":probs_with_svc,
    "LR":probs_with_logistic_regression,
    "KNN":probs_with_knn,
    "DNN":probs_with_DNN,
    "MLP":probs_with_MLPClassifier
}
#
# (0)
#
stages=[hr_st1, hr_st2]
hr_process_w_ensembles =  HRProcessWEnsembles([hr_st1, hr_st2], 'enrollee_id', base_prob_models=BASE_PROB_MODELS)

In [ ]:
%%bash
ls | grep "vf"

In [ ]:
#
# (3)
#

In [ ]:
import re
required_keys = ['negative_to_positive_ratio', 'categorical_method','scale_features', 'weights', 'threshold', 'voting_type']

In [ ]:
#
df1=pd.read_excel("Stage_1_results_vf.xlsx").sort_values(by="f1_score", ascending=False)
df1=df1[required_keys ]
df1.head()

In [ ]:
#
df2=pd.read_excel("Stage_2_results_vf.xlsx").sort_values(by="f1_score", ascending=False)
df2=df2[required_keys ]
df2.head()

In [ ]:
#
# (4)
#
def parse_weights_from_string(weight_str):
    """
    Parse the string representation of weights into a list.

    Parameters:
    weight_str (str): String representation of weights.

    Returns:
    list: List of weight percentages, with one entry per model.
    """
    # Initialize the weights vector with zeros
    weights = {model: 0 for model in BASE_PROB_MODELS}

    # Update weights based on the string
    for model_weight in weight_str.split(' + '):
        weight, model_key = model_weight.split('% ')
        weights[model_key] = float(weight) / 100

    # Return the weights in the order of the models in BASE_PROB_MODELS
    return [weights[model] for model in BASE_PROB_MODELS]

# Example of usage
weight_str = "50.0% RF + 50.0% SVC"
weights_vector = parse_weights_from_string(weight_str)
print(weights_vector)  # [0.5, 0.5, 0, 0, 0, 0] assuming the order in BASE_PROB_MODELS
#
#
#
def row_to_params_dict(row):
    """
    Convert a DataFrame row into a dictionary of parameters.

    Parameters:
    row (pd.Series): A row from the DataFrame.

    Returns:
    dict: Dictionary of parameters.
    """
    params = row.to_dict()

    # Parse weights string
    if 'weights' in params and params['weights'] is not None:
        params['weights'] = parse_weights_from_string(params['weights'])

    # Replace string representation of numeric values
    if 'threshold' in params and params['threshold'] is not None:
        params['threshold'] = float(str(params['threshold']).replace(',', '.'))

    # Handle negative_to_positive_ratio
    if 'negative_to_positive_ratio' in params:
        if pd.isna(params['negative_to_positive_ratio']):
            params['negative_to_positive_ratio'] = None
        else:
            params['negative_to_positive_ratio'] = int(params['negative_to_positive_ratio'])

    return params
#
# Apply the updated function to convert each row to a dictionary of parameters
# Convert each row to a dictionary of parameters
param_dicts1 = df1.apply(row_to_params_dict, axis=1).tolist()
param_dicts2 = df2.apply(row_to_params_dict, axis=1).tolist()


In [ ]:
#
# (5) Assuming param_dicts1 and param_dicts2 are your top configurations for stage 1 and stage 2 respectively
#
REACH_LOOK_UP_CONF_PER_STAGE = 10
results_list = []
#
for params_stage1 in param_dicts1[:REACH_LOOK_UP_CONF_PER_STAGE]:
    for params_stage2 in param_dicts2[:REACH_LOOK_UP_CONF_PER_STAGE]:
        # Create a list of parameters for each stage
        param_list = [params_stage1, params_stage2]

        # Run total_evaluation with the current combination of parameters
        result = hr_process_w_ensembles.total_evaluation(param_list)

        # Add the filtered result to the results list
        results_list.append(result)

        print(result)
#
#results_list

In [ ]:
#
# (6) Function to convert weight vector to a descriptive string
#
def parse_weights_to_string(weights):
    if isinstance(weights, str):
        return weights
    weight_strs = []
    for model_key, weight in zip(BASE_PROB_MODELS.keys(), weights):
        if weight > 0:
            weight_percent = round(weight * 100, 4)
            weight_strs.append(f"{weight_percent}% {model_key}")
    return " + ".join(weight_strs)
#
def flatten_results(all_results):
    """
    Flatten the nested results dictionaries into a list of plain dictionaries,
    excluding any 'test_ids_used' and 'final_prediction' keys.

    Parameters:
    all_results (list): List containing dictionaries of results for all stages and the total process.

    Returns:
    list: List of plain dictionaries with combined results.
    """
    flattened_results = []
    for result in all_results:
        flat_dict = {}
        for key, value in result.items():
            if key.startswith('stage'):
                stage_index = key.split('_')[1]  # Extract stage index
                for subkey, subvalue in value.items():
                    # Skip 'test_ids_used' and 'final_prediction' keys
                    if subkey.startswith('test_ids_used') or subkey.startswith('final_prediction'):
                        continue

                    # Check if this is the weight parameter to apply the parsing function
                    if subkey == 'weights' and isinstance(subvalue, list):
                        subvalue = parse_weights_to_string(subvalue)
                    new_key = f"{subkey}_{stage_index}"
                    flat_dict[new_key] = subvalue
            else:  # For 'total_process'
                flat_dict.update(value)

        flattened_results.append(flat_dict)
    return flattened_results

# Example usage
flattened_results_list = flatten_results(results_list)
flattened_results_list

In [ ]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

#
# (7)
#
df=pd.DataFrame(flattened_results_list).sort_values(by="total_f1_score", ascending=False)
df.to_excel("total_process_results.xlsx")
df.head()

#
# (8) export
#

# Assuming flattened_results_list is already defined and is a list of dictionaries
df = pd.DataFrame(flattened_results_list).sort_values(by="total_f1_score", ascending=False)

# Concatenating the two columns into a new column 'big_ensemble'
df['big_ensemble'] ="(" + df['weights_description_0'] + ")" + "+" + "(" + df['weights_description_1'] + ")"

# Reorder the DataFrame to make 'big_ensemble' the first column
cols = df.columns.tolist()
cols = ['big_ensemble'] + [col for col in cols if col != 'big_ensemble']
df = df[cols]

# Save the reordered DataFrame to an Excel file
df.to_excel("total_process_results.xlsx", index=False)

# Load the Excel file
workbook = openpyxl.load_workbook("total_process_results.xlsx")
sheet = workbook.active

# Adjust column widths
for column_cells in sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    sheet.column_dimensions[openpyxl.utils.get_column_letter(column_cells[0].column)].width = length

# Save the adjusted file
workbook.save("total_process_results.xlsx")